##1. Data Generation & Dataset Creation

###1.1. Data Loading and Preprocessing

make sure to upload md and pdf files into `/data` directory

###1.2. Synthetic Data Generation (using Qwen2.5-3B-Instruct itself, initially)

###1.3. Data Augmentation and Refinement

###1.4. Dataset Splitting

###1.5 Dataset Formatting (JSONL)

##2. Model Selection and Preparation

##3. Efficient Fine-tuning (QLoRA)